In [1]:
import requests
from lxml import etree
import csv
import re
import pandas as pd
from pathlib import Path
import time
import os


In [2]:
# target main link
url = "https://gol.gg/tournament/tournament-matchlist/LPL%20Summer%202022/"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
resp = requests.get(url, headers = headers)
resp.close()
# print(resp.text)
html = etree.HTML(resp.text)
trs = html.xpath("/html/body/div/main/div[2]/div/div[3]/div/section/div/div/table/tbody/tr")

In [13]:
def get_data(game_title, Team1, score_text, Team2, week, patch, date, game_number, match_number):
    # 要访问的小场链接
    game_url = "https://gol.gg/game/stats/"+str(int(game_number)+match_number-1)+"/page-fullstats/"
    print(game_url)
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
    sub_resp = requests.get(game_url, headers = headers)
    sub_resp.close()
    sub_html = etree.HTML(sub_resp.text)

    # 拿到表格第一行 英雄名
    champions_path = sub_html.xpath('/html/body/div/main/div[2]/div/div[3]/div/div/div/table/thead/tr/th')
    champions = ["Champions"]
    count = 0
    for item in champions_path:
        count += 1
        # print(etree.tostring(item, pretty_print=True))
        if count == 1:
            continue
        champion = item.xpath("./img/@alt")[0]
        champions.append(champion)

    df = pd.DataFrame(columns = champions)

    # 拿后面所有行的xpath
    rows = sub_html.xpath('/html/body/div/main/div[2]/div/div[3]/div/div/div/table/tr')
    # print(etree.tostring(rows[1], pretty_print=True))

    # 用for循环拿每一行的内容
    count1 = 0
    for row in rows:
        count1 += 1
        line = []
        if count1 == 1:
            line.append(row.xpath("./td[1]/text()")[0])
            for i in range(2, 12):
                content = row.xpath("./td[" + str(i)+ "]/b/text()")
                line.append(content[0])

        else:
            for i in range(1, 12):
                content = row.xpath("./td[" + str(i)+ "]/text()")
                if content == []:
                    content = [""]
                line.append(content[0])
        # 将拿到的该行内容写到dataframe里
        df.loc[len(df.index)] = line
    
    df.set_index("Champions", inplace = True)
    df = df.T
    # display(df)
    df["Game_title"] = [game_title for i in range(0,10)]
    df["Team1"] = [Team1 for i in range(0,10)]
    df["Score"] = [score_text for i in range(0,10)]
    df["Team2"] = [Team2 for i in range(0,10)]
    df["Week"] = [week for i in range(0,10)]
    df["Patch"] = [patch for i in range(0,10)]
    df["Date"] = [date for i in range(0,10)]
    df["Match_num"] = [match_number for i in range(0,10)]

    order = ["Game_title", "Team1", "Score", "Team2", "Week", "Patch", "Date", "Match_num", 
        'Player', 'Role', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS','CSM', 'Golds', 'GPM','GOLD%', 
        'WPM', 'VWPM', 'WCPM', 'Total damage to Champion', 'Physical Damage','Magic Damage', 'True Damage', 
        'DPM', 'DMG%', 'K+A Per Minute', 'KP%', 'Total damage taken', 'Level', "CS in Team's Jungle", 
        'CS in Enemy Jungle', 'Vision Score', 'Wards placed', 'Wards destroyed', 'Control Wards Purchased', 
        'Detector Wards Placed', 'VSPM', 'VS%', 'Solo kills', 'Double kills', 'Triple kills', 'Quadra kills', 
        'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15', 'Objectives Stolen', 'Damage dealt to turrets', 
        'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates', 'Damage self mitigated', 
        'Total Damage Shielded On Teammates', 'Time ccing others', 'Total Time CC Dealt', 'Total Time Spent Dead']
    df = df[order]
    # display(df)
    filepath = Path("files/lpl2022summer/full_data/" + game_title + "/" + game_title + "_match" + str(match_number) + ".csv")
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filepath, index_label = "Champion")
    
    return

# get_data("title", Team1, score_text, Team2, week, patch, date, 40250, 1)

In [14]:
# f = open("files/lpl2022summer/Summary.csv", mode = "w")
# csvwriter = csv.writer(f)
# csvwriter.writerow(["Game_title", "Team1", "Score", "Team2", "Week", "Patch", "Date"])

# 对表格中每一行的比赛链接进行提取
# 
for tr in trs[0:]:
    score_text = tr.xpath("./td[3]/text()")[0]
    # score指的是每个大场的局数，2:0 score就是2，2:1 score就是3
    score = sum(list(map(int, re.findall("\d{1}", score_text))))
    if score != 0: # 还没打的比赛也会出现在网页上，用score != 筛掉
        # 拿到表格中该行的所有信息
        link = tr.xpath("./td[1]/a/@href")[0]
        # 这个game number是网站上给每个小局赋予的一个对局编号
        # 每个大场第一个小场的编号是从该行的跳转链接里找到的
        # 对于LPL夏季赛的比赛来说，这个编号是连续的
        # 所以知道了第一个小场的编号就等于知道了第二第三个小场的编号

        game_number = re.findall("\d{5}", link)[0]
        game_title = tr.xpath("./td[1]/a/text()")[0]
        Team1 = tr.xpath("./td[2]/text()")[0]
        Team2 = tr.xpath("./td[4]/text()")[0]
        week = tr.xpath("./td[5]/text()")[0]
        patch = tr.xpath("./td[6]/text()")[0]
        date = tr.xpath("./td[7]/text()")[0]
        # csvwriter.writerow([game_title, Team1, score_text, Team2, week, patch, date])
        # print([game_title, Team1, score, Team2, week, patch, date])
        
        # 用这个for循环访问该大场的所有小场链接
        for i in range(0, score):
            get_data(game_title, Team1, score_text, Team2, week, patch, date, game_number, i+1)
            # 加sleep函数以免访问网站过于频繁造成网站服务器压力过大或者被封
            time.sleep(2)
            




https://gol.gg/game/stats/43011/page-fullstats/
https://gol.gg/game/stats/43012/page-fullstats/
https://gol.gg/game/stats/43008/page-fullstats/
https://gol.gg/game/stats/43009/page-fullstats/
https://gol.gg/game/stats/43010/page-fullstats/
https://gol.gg/game/stats/43005/page-fullstats/
https://gol.gg/game/stats/43006/page-fullstats/
https://gol.gg/game/stats/43002/page-fullstats/
https://gol.gg/game/stats/43003/page-fullstats/
https://gol.gg/game/stats/43004/page-fullstats/
https://gol.gg/game/stats/42999/page-fullstats/
https://gol.gg/game/stats/43000/page-fullstats/
https://gol.gg/game/stats/42996/page-fullstats/
https://gol.gg/game/stats/42997/page-fullstats/
https://gol.gg/game/stats/42619/page-fullstats/
https://gol.gg/game/stats/42620/page-fullstats/
https://gol.gg/game/stats/42616/page-fullstats/
https://gol.gg/game/stats/42617/page-fullstats/
https://gol.gg/game/stats/42618/page-fullstats/
https://gol.gg/game/stats/42613/page-fullstats/
https://gol.gg/game/stats/42614/page-ful

In [30]:
# 前面的是把每个大场一个单独的文件夹
# 这段就是单纯地整理成一张总表 
# 然后加了几列或者稍微改下数据格式

all_paths = os.listdir("files/lpl2022summer/full_data")
all_paths.remove(".DS_Store")
df = ''
for i in range(0, len(all_paths)):
    all_sub_paths = os.listdir("files/lpl2022summer/full_data/"+all_paths[i])
    for j in range(0, len(all_sub_paths)):
        if i == 0 and j == 0:
            df = pd.read_csv("files/lpl2022summer/full_data/" + all_paths[i] + "/" + all_sub_paths[j])
        else:
            new_df = pd.read_csv("files/lpl2022summer/full_data/" + all_paths[i] + "/" + all_sub_paths[j])
            df = pd.concat([df, new_df], ignore_index = True)
df.rename(columns={'Unnamed: 0':'Champions'}, inplace=True)
df["DMG/GOLD"] = df["Total damage to Champion"]/df["Golds"]
df['DMG%'] = df['DMG%'].str.rstrip('%').astype('float') / 100.0
df['KP%'] = df['KP%'].str.rstrip('%').astype('float') / 100.0
df['GOLD%'] = df['GOLD%'].str.rstrip('%').astype('float') / 100.0
df['K+A'] = df["Kills"] + df["Assists"]
order = ["Champion", "Game_title", "Team1", "Score", "Team2", "Week", "Patch", "Date", "Match_num", 'Player', 'Role', 
        'Kills', 'Deaths', 'Assists', 'KDA', 'K+A', 'K+A Per Minute', 'KP%', 'CS','CSM', 'Golds', 'GPM', 
        'GOLD%', 'Total damage to Champion', 'Physical Damage','Magic Damage', 
        'True Damage', 'DPM', 'DMG%', "DMG/GOLD", 'Total damage taken', 'Level', "CS in Team's Jungle", 
        'CS in Enemy Jungle', 'Vision Score', 'Wards placed', 'Wards destroyed', 'Control Wards Purchased', 
        'Detector Wards Placed', 'VSPM', 'VS%', 'Solo kills', 'Double kills', 'Triple kills', 'Quadra kills', 
        'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15', 'Objectives Stolen', 'WPM', 'VWPM', 'WCPM', 'Damage dealt to turrets', 
        'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates', 'Damage self mitigated', 
        'Total Damage Shielded On Teammates', 'Time ccing others', 'Total Time CC Dealt', 'Total Time Spent Dead']
df = df[order]
df.sort_values(by = ["Date", "Match_num"], ascending = True, inplace = True, axis = 0, ignore_index = True)
df.to_csv("files/lpl2022summer/Total.csv")
display(df)

,Champion,Game_title,Team1,Score,Team2,Week,Patch,Date,Match_num,Player,...,WCPM,Damage dealt to turrets,Damage dealt to buildings,Total heal,Total Heals On Teammates,Damage self mitigated,Total Damage Shielded On Teammates,Time ccing others,Total Time CC Dealt,Total Time Spent Dead
0,Camille,EDG vs FPX,Funplus Phoenix,1 - 2,Edward Gaming,WEEK1,12.10,2022-06-10,1,Xiaolaohu,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Viego,EDG vs FPX,Funplus Phoenix,1 - 2,Edward Gaming,WEEK1,12.10,2022-06-10,1,Clid,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Taliyah,EDG vs FPX,Funplus Phoenix,1 - 2,Edward Gaming,WEEK1,12.10,2022-06-10,1,Care,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kalista,EDG vs FPX,Funplus Phoenix,1 - 2,Edward Gaming,WEEK1,12.10,2022-06-10,1,Lwx,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sett,EDG vs FPX,Funplus Phoenix,1 - 2,Edward Gaming,WEEK1,12.10,2022-06-10,1,Hang,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,Gwen,LGD Gaming vs TT,LGD Gaming,1 - 2,TT,WEEK9,12.13,2022-08-03,3,xiaoxiang,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2896,Wukong,LGD Gaming vs TT,LGD Gaming,1 - 2,TT,WEEK9,12.13,2022-08-03,3,Beichuan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2897,Sylas,LGD Gaming vs TT,LGD Gaming,1 - 2,TT,WEEK9,12.13,2022-08-03,3,Ucal,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2898,Sivir,LGD Gaming vs TT,LGD Gaming,1 - 2,TT,WEEK9,12.13,2022-08-03,3,Kepler,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
